In [19]:
import numpy as np
import pandas as pd
import scipy.spatial as ss
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import time

import warnings
warnings.filterwarnings("ignore")

<h2> Supported section </h2>

In [20]:
def create_weight_matrix(X,Y):
    matrix = np.zeros([X.shape[0],Y.shape[0]])
    for i in range(len(X)):
        for j in range(len(Y)):
            matrix[i][j] = ss.distance.euclidean([X.iloc[i],Y.iloc[i]], [X.iloc[j],Y.iloc[j]])
    return matrix


def open_file(name_folder, name_file):
    arr = []
    with open(f'./{name_folder}/{name_file}.txt', 'r') as file:
        for line in file:
            arr.append(line.split())
    df = pd.DataFrame(arr, columns=['CUST_NO', 'XCOORD', 'YCOORD', 'DEMAND', 'READY_TIME', 'DUE_DATE',
           'SERVICE_TIME']).astype(np.int64)
    return df, arr

In [21]:
def F(route,weights_matrix,e,l,service,al1,al2,al3):
    res = 0
    for i in range(len(route) - 1):
        t = al1*weights_matrix[route[i],route[i+1]]
        ser = service[i]
        load_e = al2*max(0,e[i] - (res + t))
        load_l = al3*max(0,(res + t) - l[i])
        res += (t + ser + load_e+load_l)
    res += weights_matrix[route[-1],0]
    return res


def calc_dist(route, weights_matrix):
    s = 0
    for i in range(len(route) - 1):
        s += weights_matrix[route[i],route[i+1]]
    s += weights_matrix[route[-1],0]
    return s


def calc_distance(routes, weight_matrix):
    s = 0
    for route in routes:
        for i in range(len(route) - 1):
            s += weight_matrix[route[i],route[i+1]]
        s += weight_matrix[route[-1],0]
    return s

<h2> Clustering section </h2>

In [22]:
def clust_optim(df_copy, weight_matrix):
    cls = df_copy.groupby(by='CLUSTER',as_index=False).sum()[['CLUSTER','DEMAND']][df_copy.groupby(by='CLUSTER',as_index=False).sum()[['CLUSTER','DEMAND']]['DEMAND'] > 200]['CLUSTER'].to_list()
    flag = False
    ind_cls = 0
    iter_ind = 0
    while len(cls) > 0 and iter_ind < 30 and not flag:
        pas_clus = cls[ind_cls]
        list_with_min_pair = []
        pas_ver = []
        for customer in df_copy[df_copy['CLUSTER'] == pas_clus]['CUST_NO']:
            matrix = weight_matrix[customer]
            cur_demand = df_copy.at[customer,'DEMAND']
            min_point, min_value = -1, np.inf
            for ind in range(len(matrix)):
                if ind == 0:
                    continue
                ind_cur_clus = df_copy.at[ind,'CLUSTER']
                if ind != customer and matrix[ind] < min_value and ind_cur_clus != pas_clus and df_copy[df_copy['CLUSTER'] == ind_cur_clus]['DEMAND'].sum() <= (200 - cur_demand) and ind not in pas_ver:
                    min_point, min_value = ind, matrix[ind]
            list_with_min_pair.append((customer,min_point,min_value))
            pas_ver.append(min_point)
        arr_sorted = sorted(list_with_min_pair, key=lambda x: x[2])
        for elem_ind in range(len(arr_sorted)):
            if arr_sorted[elem_ind][1] == -1:
                continue
#            if df_copy[df_copy['CLUSTER'] == df_copy.at[arr_sorted[elem_ind][1],'CLUSTER']]['DEMAND'].sum() <= (200 - df_copy.at[arr_sorted[elem_ind][0], 'DEMAND']):
            df_copy.at[arr_sorted[elem_ind][0],'CLUSTER'] = df_copy.at[arr_sorted[elem_ind][1],'CLUSTER']
            if df_copy[df_copy['CLUSTER'] == pas_clus]['DEMAND'].sum() <= 200:
                break
        if df_copy[df_copy['CLUSTER'] == pas_clus]['DEMAND'].sum() > 200:
            flag = True
        cls = df_copy.groupby(by='CLUSTER',as_index=False).sum()[['CLUSTER','DEMAND']][df_copy.groupby(by='CLUSTER',as_index=False).sum()[['CLUSTER','DEMAND']]['DEMAND'] > 200]['CLUSTER'].to_list()
        iter_ind += 1
    if flag:
        return True
    return df_copy



def clustering(df,vehicle_number,capacited,arr,weight_matrix):
    X = df.values[1:,[1,2]]
    iter_cluster = 600
    i = 0
    while i < iter_cluster:
        k_means = KMeans(init='k-means++', n_clusters=vehicle_number, n_init=2)
        k_means.fit(X)
        labels = k_means.labels_
        df_copy = pd.DataFrame(arr, columns=['CUST_NO', 'XCOORD', 'YCOORD', 'DEMAND', 'READY_TIME', 'DUE_DATE',
           'SERVICE_TIME']).astype(np.int64)
        df_copy['CLUSTER'] = [np.nan,*labels]
        if np.all((df_copy.groupby(by='CLUSTER')['DEMAND'].sum() <= capacited).to_list()):
            return [np.nan,*labels]
            break
        i += 1
        if i > 400:
            feat = clust_optim(df_copy, weight_matrix)
            if type(feat) != bool:
                df_copy = feat
                if np.all((df_copy.groupby(by='CLUSTER')['DEMAND'].sum() <= capacited).to_list()):
                    return df_copy['CLUSTER'].to_list()
        if i == iter_cluster:
            vehicle_number += 1
            print(vehicle_number)
            i = 0
    return None

<h2> Greedy algorythm section </h2>

In [23]:
def open_pair(pairs, passed_vertex):
    arr = []
    for pair in pairs:
        if pair[0] not in passed_vertex:
            arr.append(pair)
    return arr

In [24]:
def greedy_algorythm(df,weights_matrix,k,vehicle_number):
    costs = []
    routes = []
    for ind in range(vehicle_number):
        df_cluster = pd.concat([df[df['CUST_NO'] == 0],df[df['CLUSTER'] == ind]])
        customers = df_cluster['CUST_NO']
        e = df_cluster['READY_TIME']
        l = df_cluster['DUE_DATE']
        ready_pair = [(customer,ready_time) for customer,ready_time in zip(customers,e)]
        passed_vertexes = [0]
        current_vertex = 0
        distance = 0
        route = [0]
        while len(passed_vertexes) != len(customers):
            sorted_vertexes_pair = sorted(open_pair(ready_pair,passed_vertexes), key=lambda x: x[1])[:k]
            min_value = np.inf
            min_pair = 0
            for pair in sorted_vertexes_pair:
                if weights_matrix[current_vertex][pair[0]] < min_value:
                    min_pair = pair
                    min_value = weights_matrix[current_vertex][pair[0]]
            passed_vertexes.append(min_pair[0])
            distance += weights_matrix[current_vertex][min_pair[0]]
            current_vertex = min_pair[0]
            route.append(current_vertex)
        costs.append(distance)
        routes.append(route)
    s = 0
    for route in routes:
        s1 = 0
        for i in range(len(route) - 1):
            s1 += weights_matrix[route[i],route[i+1]]
        s1 += weights_matrix[route[-1],0]
        s += s1
    return routes, s

<h2> ACO section </h2>

In [25]:
def arr_cut(arr, ver):
    result = []
    for ind in range(len(arr)):
        if ind not in ver:
            result.append(arr[ind])
    return np.array(result)

In [26]:
def run_single_ant(numbers, weights_matrix, pheromons_matrix, service, e, l):
    alpha = 5
    beta = 2
    lambda1 = 0.4
    lambda2 = 0.4
    lambda3 = 0.2
    current_vertex = 0
    index_vertex = [i for i in range(len(numbers))]
    passed_vertex_ind = [0]
    passed_vertex = [0]
    while len(passed_vertex) != len(numbers):
        pheromons_current = arr_cut(pheromons_matrix[current_vertex], passed_vertex_ind)
        weights_current = arr_cut(weights_matrix[current_vertex], passed_vertex_ind)
        e_current = arr_cut(e, passed_vertex_ind)
        l_current = arr_cut(l, passed_vertex_ind)
        tau = pheromons_current**alpha
        nu1 = (np.max(weights_current) - weights_current) / (np.max(weights_current) - np.min(weights_current))
        nu2 = (np.max(e_current) - e_current) / (np.max(e_current) - np.min(e_current))
        nu3 = (np.max(l_current) - l_current) / (np.max(l_current) - np.min(l_current))
        if (np.max(weights_current) - np.min(weights_current)) == 0:
            nu1 = np.array([1 for _ in range(len(nu1))])
        if (np.max(e_current) - np.min(e_current)) == 0:
            nu2 = np.array([1 for _ in range(len(nu1))])
        if (np.max(l_current) - np.min(l_current)) == 0:
            nu3 = np.array([1 for _ in range(len(nu1))])
        nu = lambda1*nu1 + lambda2*nu2 + lambda3*nu3
        prob = (tau**alpha*nu**beta) / (tau**alpha*nu**beta).sum()
        choose = arr_cut(index_vertex, passed_vertex_ind)
        if len(choose) == 1:
            choose_vertex = choose[0]
        else:
            choose_vertex = random.choices(choose, weights=prob)[0]
        passed_vertex_ind.append(choose_vertex)
        passed_vertex.append(numbers[choose_vertex])
        current_vertex = choose_vertex
        
    return passed_vertex_ind, passed_vertex

In [27]:
def ACO(df,vehicle_number,e_main,l_main,service,weight_matrix,al1,al2,al3):
    cost = []
    routes = []
    best_routes, best_costs = [], []
    for ind in range(vehicle_number):
        df_cluster = pd.concat([df[df['CUST_NO'] == 0],df[df['CLUSTER'] == ind]])
        customers_clus = df_cluster['CUST_NO'].to_list()
        service_clus = df_cluster['SERVICE_TIME'].to_list()
        pheromons_matrix = np.zeros([df_cluster.shape[0], df_cluster.shape[0]]) + 0.3
        weights_matrix = create_weight_matrix(df_cluster['XCOORD'],df_cluster['YCOORD'])
        e,l = df_cluster['READY_TIME'].to_list(),df_cluster['DUE_DATE'].to_list()
        
        #if df_cluster.shape[0] < 5:
        #    max_iter = 100
        #elif df_cluster.shape[0] < 10:
        #    max_iter = 200
        #elif df_cluster.shape[0] < 15:
        #    max_iter = 300
        #else:
        #    max_iter = 400
        max_iter = 100
        colony_size = 15
        dispersion_coef = 0.05
        Q = 3
        best_route = []
        best_cost = np.inf
        
        for i in range(max_iter):
            # create solution ant
            ants = []
            ants_ind = []
            for j in range(colony_size):
                ant_ind, ant = run_single_ant(customers_clus,weights_matrix,pheromons_matrix,service_clus,e,l)
                ants_ind.append(ant_ind)
                ants.append(ant)

            # search best solution ant
            best_sol = np.inf
            best_ind = -1
            for j in range(colony_size):
                if F(ants[j], weight_matrix,e_main,l_main,service,al1,al2,al3) < best_sol:
                    best_sol = F(ants[j], weight_matrix,e_main,l_main,service,al1,al2,al3)
                    best_ind = j
            if best_sol < best_cost:
                best_cost = best_sol
                best_route = ants[best_ind]

            # update pheromons
            pheromons_matrix = (1 - dispersion_coef) * pheromons_matrix
            for f in range(len(ants)):
                #len_route = calc_dist(ants[f], weight_matrix)
                len_route = F(ants[f], weight_matrix,e_main,l_main,service,al1,al2,al3)
                if f == best_ind:
                    for j in range(len(ants[f])-1):
                        pheromons_matrix[ants_ind[f][j]][ants_ind[f][j+1]] += Q / len_route
                else:
                    for j in range(len(ants[f])-1):
                        pheromons_matrix[ants_ind[f][j]][ants_ind[f][j+1]] += 1 / len_route
        ant_ind, ant = run_single_ant(customers_clus,weights_matrix,pheromons_matrix,service_clus,e,l)
        
        cost.append(calc_dist(best_route, weight_matrix))
        routes.append(best_route)
    return routes, cost

<h2> Tabu search section </h2>

In [28]:
def tabu_search(routes,weights_matrix,e,l,service,al1,al2,al3):
    new_cost = []
    new_routes=  []
    for i in range(len(routes)):
        current = routes[i].copy()
        if len(current) <= 2:
            new_cost.append(calc_dist(current,weights_matrix))
            new_routes.append(current)
            continue
        elif len(current) == 3:
            new_route = current.copy()
            new_route[1], new_route[2] = new_route[2], new_route[1]
            if F(current,weights_matrix,e,l,service,al1,al2,al3) > F(new_route, weights_matrix,e,l,service,al1,al2,al3):
                new_cost.append(calc_dist(new_route,weights_matrix))
                new_routes.append(new_route)
            else:
                new_cost.append(calc_dist(current,weights_matrix))
                new_routes.append(current)
            continue
        best_route = current.copy()
        cur_iter = 0
        max_iter = 4000
        max_size_tabu = np.max([1,len(current) // 2])
        C = np.max([1,len(current) // 3])
        tabu_list = []
        while cur_iter < max_iter:
            possible_move = []
            iter_C = 0
            it_C = 0
            while iter_C < C and it_C < 100:
                choices = random.sample(current[1:],k=2)
                a,b = current.index(choices[0]), current.index(choices[1])
                current_1 = current.copy()
                current_1[b], current_1[a] = current_1[a], current_1[b]
                if F(current,weights_matrix,e,l,service,al1,al2,al3) < F(best_route,weights_matrix,e,l,service,al1,al2,al3):
                    best_route = current.copy()
                    iter_C = 0
                    continue
                if (a,b) not in tabu_list and (b,a) not in tabu_list:
                    new_route = current.copy()
                    new_route[b], new_route[a] = current[a], current[b]
                    possible_move.append(((a,b),F(new_route,weights_matrix,e,l,service,al1,al2,al3)))
                    iter_C += 1
                it_C += 1
            if len(possible_move) == 0:
                cur_iter += 1
                continue
            loc_best = sorted(possible_move, key=lambda x: x[1])[0]
            if F(current,weights_matrix,e,l,service,al1,al2,al3) > loc_best[1]:
                current[loc_best[0][0]], current[loc_best[0][1]] = current[loc_best[0][1]], current[loc_best[0][0]]
            tabu_list.append(loc_best[0])
#            choices = random.sample(current[1:],k=2)
#            a,b = current.index(choices[0]), current.index(choices[1])
#            new_route = current.copy()
#            new_route[b], new_route[a] = current[a], current[b]
#            if F(new_route,weights_matrix,e,l,service,al1,al2,al3) < F(best_route,weights_matrix,e,l,service,al1,al2,al3):
#                current[a], current[b] = current[b], current[a]
#                best_route = new_route.copy()
#                cur_iter = 0
#            else:
#                if (a,b) not in tabu_list and (b,a) not in tabu_list:
#                    if F(new_route,weights_matrix,e,l,service,al1,al2,al3) < F(current,weights_matrix,e,l,service,al1,al2,al3):
#                        current[a], current[b] = current[b], current[a]
#                        tabu_list.append((a,b))
#                        cur_iter = 0
            if len(tabu_list) > max_size_tabu:
                tabu_list = tabu_list[1:]
            cur_iter += 1
        new_cost.append(calc_dist(best_route,weights_matrix))
        new_routes.append(best_route)
    return new_routes, new_cost

<h1> MAIN SECTION </h1>

In [37]:
path_file = 'files_C'
name_file = 'c101'
capacited = 200
vehicle_number = 10
df, arr = open_file(path_file,name_file)
weight_matrix = create_weight_matrix(df['XCOORD'],df['YCOORD'])
e, l, service = df['READY_TIME'].to_list(), df['DUE_DATE'].to_list(), df['SERVICE_TIME'].to_list()

In [38]:
g, a, t = [], [], []
for i in tqdm(range(3)):
    while True:
        feat = clustering(df, vehicle_number, capacited, arr,weight_matrix)
        if feat != None:
            df['CLUSTER'] = feat
            break
    greed = greedy_algorythm(df,weight_matrix,1,vehicle_number)
    route_ACO, cost_ACO = ACO(df,vehicle_number,e,l,service,weight_matrix,1,0,0)
    r1, r2 = tabu_search(route_ACO,weight_matrix,e,l,service,1,0,0)
    g.append(np.around(greed[1],1))
    a.append(np.around(np.sum(cost_ACO),1))
    t.append(np.around(np.sum(r2),1))
    
print('greedy')
print(g)
print('ACO')
print(a)
print('TS')
print(t)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:26<00:00, 28.80s/it]

greedy
[828.9, 828.9, 828.9]
ACO
[831.8, 841.2, 832.8]
TS
[828.9, 830.9, 829.6]
